In [1]:
pip install pennylane scikit-learn pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pennylane as qml
from pennylane import numpy as np

# -------------------- Load & Preprocess Dataset --------------------

df = pd.read_csv("/Users/neerajprao/Downloads/ML-vs-DL-main/datasets/predict-bank-churn.csv")  # Replace with your actual file path

# One-hot encoding
df = pd.get_dummies(df, columns=["Geography", "Gender"])

# Drop unnecessary columns
df.drop(["id", "CustomerId", "Surname"], axis=1, inplace=True)

# Feature list
features = ['CreditScore', 'Geography_France', 'Geography_Germany', 'Geography_Spain',
            'Gender_Female', 'Gender_Male', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
            'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
X = df[features]
y = df["Exited"].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PennyLane numpy arrays
X_train = np.array(X_train, requires_grad=False)
X_test = np.array(X_test, requires_grad=False)
y_train = np.array(y_train, requires_grad=False)
y_test = np.array(y_test, requires_grad=False)

# -------------------- Quantum Model Definition --------------------

n_qubits = 13
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))  # Output from qubit 0

def variational_classifier(inputs, weights):
    return circuit(inputs, weights)

def square_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def accuracy(y_true, y_pred):
    predictions = np.where(y_pred >= 0.5, 1, 0)
    return np.mean(predictions == y_true)

# -------------------- Training --------------------

from pennylane.optimize import AdamOptimizer

weights = np.random.randn(2, n_qubits, requires_grad=True)
opt = AdamOptimizer(stepsize=0.05)
batch_size = 16
epochs = 20

for epoch in range(epochs):
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        def cost(weights):
            preds = [variational_classifier(x, weights) for x in X_batch]
            return square_loss(y_batch, preds)

        weights = opt.step(cost, weights)

    # Evaluation
    y_preds = [variational_classifier(x, weights) for x in X_test]
    acc = accuracy(y_test, y_preds)
    print(f"Epoch {epoch+1}/{epochs} - Accuracy: {acc:.4f}")

# -------------------- Final Evaluation --------------------

final_preds = [1 if p >= 0.5 else 0 for p in y_preds]
print("\nFinal Accuracy:", accuracy_score(y_test, final_preds))


/opt/anaconda3/envs/yolov8-env/lib/python3.10/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


TypeError: '>=' not supported between instances of 'list' and 'float'

In [5]:
# Ensure y_preds is a list of scalars (probabilities)
y_preds = [variational_classifier(x, weights) for x in X_test]
y_preds = [float(pred) for pred in y_preds]  # Convert each prediction to a scalar

# Calculate final accuracy
final_preds = [1 if p >= 0.5 else 0 for p in y_preds]
print("\nFinal Accuracy:", accuracy_score(y_test, final_preds))

# Optionally, print confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report

print("Confusion Matrix:")
print(confusion_matrix(y_test, final_preds))

print("\nClassification Report:")
print(classification_report(y_test, final_preds))



Final Accuracy: 0.7892871209137455
Confusion Matrix:
[[    0     0     0]
 [    0 26052     0]
 [    0  6955     0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      1.00      0.88     26052
           0       0.79      1.00      0.88     26052
           1       0.00      0.00      0.00      6955

    accuracy                           0.83     59059
   macro avg       0.53      0.67      0.59     59059
weighted avg       0.70      0.88      0.78     59059



/opt/anaconda3/envs/yolov8-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/yolov8-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/envs/yolov8-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

In [6]:
# Save the trained weights to a file
np.save("quantum_model_weights.npy", weights)

# To load them later (if needed):
# weights = np.load("quantum_model_weights.npy", allow_pickle=True)


Accuracy = 0.78